In [1]:
from __future__ import print_function
from distutils.version import LooseVersion as Version
import sys

OK = '\x1b[42m[ OK ]\x1b[0m'
FAIL = "\x1b[41m[FAIL]\x1b[0m"

try:
    import importlib
except ImportError:
    print(FAIL, "Python version 3.7 is required,"
                " but %s is installed." % sys.version)
def import_version(pkg, min_ver, fail_msg=""):
    mod = None
    try:
        mod = importlib.import_module(pkg)
        if pkg in {'PIL'}:
            ver = mod.VERSION
        else:
            ver = mod.__version__
        if Version(ver) == min_ver:
            print(OK, "%s version %s is installed."
                % (lib, min_ver))
        else:
            print(FAIL, "%s version %s is required, but %s installed."
                % (lib, min_ver, ver))
    except ImportError:
        print(FAIL, '%s not installed. %s' % (pkg, fail_msg))
    return mod

# first check the python version
pyversion = Version(sys.version)
if pyversion >= "3.7":
    print(OK, "Python version is %s" % sys.version)
elif pyversion < "3.7":
    print(FAIL, "Python version 3.7 is required,"
                " but %s is installed." % sys.version)
else:
    print(FAIL, "Unknown Python version: %s" % sys.version)

    
print()
requirements = {'numpy': "1.18.5", 'matplotlib': "3.2.2",'sklearn': "0.23.1",
                'pandas': "1.0.5",'xgboost': "1.1.1", 'shap': "0.35.0"}
# now the dependencies
for lib, required_version in list(requirements.items()):
    import_version(lib, required_version)

[ OK ] Python version is 3.7.4 (v3.7.4:e09359112e, Jul  8 2019, 14:54:52) 
[Clang 6.0 (clang-600.0.57)]

[ OK ] numpy version 1.18.5 is installed.
[ OK ] matplotlib version 3.2.2 is installed.
[ OK ] sklearn version 0.23.1 is installed.
[ OK ] pandas version 1.0.5 is installed.
[ OK ] xgboost version 1.1.1 is installed.
[ OK ] shap version 0.35.0 is installed.


In [2]:
# reading file
# sep = '\s+'  means any amount of white space as delimeter

import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from matplotlib import pyplot as plt

# df = pd.read_csv("diabetes-data.csv")
df =  pd.read_csv('diabetes-data.csv', sep='\s+'  , engine='python')
# ids = df["ResponseId"]

print('Questions:')
print()
print('1. Is the dataset IID or not?')
# IID: Independent and Identically Distributed
    # all samples come from same generative process and have no memory of past generated samples
            # identify cats and dogs on images
            # predict the house price
            # predict if someone's salary is above or below 50k
# non-IID:
    #data generated by time-dependent processes
            # data has group structure 
            # (samples collected from e.g., different subjects, experiments, measurement devices)
            # stock price, youtube video stats, returning customer spending
            # multiple data points that belong to the same subject or same group

print('The dataset is IID because the data does not depend on each other. It has no memory of past samples.')
print('Each row is not related to each other.')
print()
print('2. Please decide what fraction of points will be in each set and explain your decision.')
print('answer')
print('Its usually safe to do 60% train, 20% validation, 20% test for small datasets.')
print()
print('3. Please explain in a paragraph or two why it is important to fit the preprocessors on the training set only.')
print('Data very rarely comes in the proper format for machine learning. ' 
      'Often it contains many strings that are not numerical values.'
      ' You need to standardize all of the features so the mean=0, stdev=1.'
        ' Difficult to do analysis when data is not normalized yet.')
df.head()

Questions:

1. Is the dataset IID or not?
The dataset is IID because the data does not depend on each other. It has no memory of past samples.
Each row is not related to each other.

2. Please decide what fraction of points will be in each set and explain your decision.
answer
Its usually safe to do 60% train, 20% validation, 20% test for small datasets.

3. Please explain in a paragraph or two why it is important to fit the preprocessors on the training set only.
Data very rarely comes in the proper format for machine learning. Often it contains many strings that are not numerical values. You need to standardize all of the features so the mean=0, stdev=1. Difficult to do analysis when data is not normalized yet.


,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,59,2,32.1,101.0,157,93.2,38.0,4.0,4.8598,87,151
1,48,1,21.6,87.0,183,103.2,70.0,3.0,3.8918,69,75
2,72,2,30.5,93.0,156,93.6,41.0,4.0,4.6728,85,141
3,24,1,25.3,84.0,198,131.4,40.0,5.0,4.8903,89,206
4,50,1,23.0,101.0,192,125.4,52.0,4.0,4.2905,80,135


In [3]:
# Basic Split (Train-test)

from sklearn.model_selection import train_test_split 


# help(train_test_split)

random_state = 42

# let's separate the feature matrix X, and target variable y
y = df['BMI'] # remember, we want to predict BMI
X = df.loc[:, df.columns != 'BMI'] # all other columns are features
# print(y)
# print(X.head())

# first split to separate out the training set
# 60% train
X_train, X_other, y_train, y_other = train_test_split(X,y,train_size = 0.6,random_state=random_state)
print('training set:',X_train.shape, y_train.shape) # 60% of points are in train
print(X_other.shape, y_other.shape) # 40% of points are in other

# second split to separate out the validation and test sets
# 40% test; (20% validation 20% test)
X_val, X_test, y_val, y_test = train_test_split(X_other,y_other,train_size = 0.5,random_state=random_state)
print('validation set:',X_val.shape, y_val.shape) # 20% of points are in validation
print('test set:',X_test.shape, y_test.shape) # 20% of points are in test

# IMPORTANT: RANDOMNESS
# there is always a randomness in splitting, depending on which points are in train validation vs. tests
# this is a constant noise that occurs
# as a result, important to run this a few times and calculate the mean + stdev of the test scores

training set: (265, 10) (265,)
(177, 10) (177,)
validation set: (88, 10) (88,)
test set: (89, 10) (89,)


In [4]:
# Data Preprocessing
    # Categorical: use ordinal or one-hot encoding
    # Continuous: Scaling and normalization

# STEP 1: Use transformers
    
    # All transformes have three methods:
        # fit method: estimates parameters necessary to do the transformation,
        # transform method: transforms the data based on the estimated parameters,
        # fit_transform method: both steps are performed at once, this can be faster than doing the steps separately.
        
    # Transformers we cover in class:
        # OneHotEncoder - converts categorical features into dummy arrays
        # OrdinalEncoder - converts categorical features into an integer array
        # MinMaxScaler - scales continuous variables to be between 0 and 1
        # StandardScaler - standardizes continuous features by removing the mean and scaling to unit variance
        # LabelEncoder - converts text target variable to numerical values
        

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,59,2,32.1,101.0,157,93.2,38.0,4.0,4.8598,87,151
1,48,1,21.6,87.0,183,103.2,70.0,3.0,3.8918,69,75
2,72,2,30.5,93.0,156,93.6,41.0,4.0,4.6728,85,141
3,24,1,25.3,84.0,198,131.4,40.0,5.0,4.8903,89,206
4,50,1,23.0,101.0,192,125.4,52.0,4.0,4.2905,80,135


In [11]:
# Ordinal Encoding
    # Use in features where categories can be ranked or ordered
    # Education level, reaction "severe", "response", "excellent"
    

from sklearn.preprocessing import OrdinalEncoder


# example data
train_edu = {'educational level':['Bachelors','Masters','Bachelors','Doctorate','HS-grad','Masters']} 
test_edu = {'educational level':['HS-grad','Masters','Masters','College','Bachelors']}

Xtoy_train = pd.DataFrame(train_edu)
Xtoy_test = pd.DataFrame(test_edu)


# initialize the encoder -- manual ordered list of categories
# the string 'HS-grad' will be replaced by 0, Doctorate will be #4
cats = [['HS-grad','College','Bachelors','Masters','Doctorate']]
enc = OrdinalEncoder(categories = cats) # The ordered list of 
# categories need to be provided. By default, the categories are alphabetically ordered!


# fit the training data
enc.fit(Xtoy_train)
print(enc.categories_)

# transform X_train and X_test
# We could have used enc.fit_transform(X_train) to combine fit and transform
X_train_oe = enc.transform(Xtoy_train)
print('transformed train features: ')
print(X_train_oe)
X_test_oe = enc.transform(Xtoy_test) # OrdinalEncoder error if it encounters an unknown category in test
print('transformed test features: ')
print(X_test_oe)

[array(['HS-grad', 'College', 'Bachelors', 'Masters', 'Doctorate'],
      dtype=object)]
transformed train features: 
[[2.]
 [3.]
 [2.]
 [4.]
 [0.]
 [3.]]
transformed test features: 
[[0.]
 [3.]
 [3.]
 [1.]
 [2.]]


In [13]:
# One hot Encoder -- ordinal
    # use if categories cannot be clearly ordered (gender, workclass, relationship status)
    # converts 1 categorical column into multiple columns
    # if feature has X category, then it's 1. otherwise it's 0. (Female =1, male=0, other=0)

from sklearn.preprocessing import OneHotEncoder
# help(OneHotEncoder)

# toy example
train = {'gender':['Male','Female','Unknown','Male','Female','Female'],\
         'browser':['Safari','Safari','Internet Explorer','Chrome','Chrome','Internet Explorer']}
test = {'gender':['Female','Male','Unknown','Female'],'browser':['Chrome','Firefox','Internet Explorer','Safari']}

Xtoy_train = pd.DataFrame(train)
Xtoy_test = pd.DataFrame(test)

ftrs = ['gender','browser']


# initialize the encoder
enc = OneHotEncoder(sparse=False,handle_unknown='ignore') # by default, OneHotEncoder returns a sparse matrix. 
                                                          # sparse=False returns a 2D array

# fit the training data
enc.fit(Xtoy_train)
print('categories:',enc.categories_)
print('feature names:',enc.get_feature_names(ftrs))

# transform X_train
X_train_ohe = enc.transform(Xtoy_train)
#print(X_train_ohe)

# do all of this in one step
X_train_ohe = enc.fit_transform(Xtoy_train)
print('X_train transformed')
print(X_train_ohe)

# transform X_test
X_test_ohe = enc.transform(Xtoy_test)
print('X_test transformed')
print(X_test_ohe)

categories: [array(['Female', 'Male', 'Unknown'], dtype=object), array(['Chrome', 'Internet Explorer', 'Safari'], dtype=object)]
feature names: ['gender_Female' 'gender_Male' 'gender_Unknown' 'browser_Chrome'
 'browser_Internet Explorer' 'browser_Safari']
X_train transformed
[[0. 1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 1. 0.]]
X_test transformed
[[1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0.]
 [1. 0. 0. 0. 0. 1.]]


In [20]:
# Min Max Scaler -- continuous
    # Continuous features that are reasonably bounded (e.g., age (0-100), # hrs worked per week (0-80))
    # Plot the histogram feature to verify if needed
    # do not refit Min Max Scaler on test set
    

# numbers are automatically put between 0 and 1
# this is automatically done in the minmaxscaler; it does it like this:
# (value - min) / (max - min), if value is 32, min is 13 and max is 75, then we have 19 / 62 = 0.3064


from sklearn.preprocessing import MinMaxScaler
# help(MinMaxScaler)


# toy data
# let's assume we have two continuous features:
train = {'age':[32,65,13,68,42,75,32],'number of hours worked':[0,40,10,60,40,20,40]}
test = {'age':[83,26,10,60],'number of hours worked':[0,40,0,60]}


Xtoy_train = pd.DataFrame(train)
Xtoy_test = pd.DataFrame(test)

# initialize and fit the scaler
scaler = MinMaxScaler()
scaler.fit(Xtoy_train)
print('transformed data:')
print(scaler.transform(Xtoy_train))
print()
print('train data:')
print(scaler.transform(Xtoy_test)) # note how scaled X_test contains values larger than 1 and smaller than 0.

transformed data:
[[0.30645161 0.        ]
 [0.83870968 0.66666667]
 [0.         0.16666667]
 [0.88709677 1.        ]
 [0.46774194 0.66666667]
 [1.         0.33333333]
 [0.30645161 0.66666667]]

train data:
[[ 1.12903226  0.        ]
 [ 0.20967742  0.66666667]
 [-0.0483871   0.        ]
 [ 0.75806452  1.        ]]


In [25]:
# Standard Scaler -- continuous
    # Use if continuous feature follows and tailed distribution
    # E.g., salaries (most people earn <100k but there are some super rich people)
    # mean =0, stdev=1
    
from sklearn.preprocessing import StandardScaler
# help(StandardScaler)

# toy data
train = {'salary':[50_000,75_000,40_000,1_000_000,30_000,250_000,35_000,45_000]}
test = {'salary':[25_000,55_000,1_500_000,60_000]}

Xtoy_train = pd.DataFrame(train)
Xtoy_test = pd.DataFrame(test)

scaler = StandardScaler()
print('train data: ')
print(scaler.fit_transform(Xtoy_train))
print()
print('test data: ')
print(scaler.transform(Xtoy_test))

train data: 
[[-0.44873188]
 [-0.36895732]
 [-0.4806417 ]
 [ 2.58270127]
 [-0.51255153]
 [ 0.18946457]
 [-0.49659661]
 [-0.46468679]]

test data: 
[[-0.52850644]
 [-0.43277697]
 [ 4.1781924 ]
 [-0.41682206]]


In [27]:
# Label Encoder
    # Classification labels need to be integers with values between 0 and n_classes-1
    # Labels should be 0, 1 or if there are 3 classes, then 0, 1, 2.
    
from sklearn.preprocessing import LabelEncoder
# help(LabelEncoder)

# toy data
ytoy_train = ['>50K','>50K', '>50K', '<=50K', '<=50K', '>50K']
ytoy_test = ['<=50K','>50K','>50K','<=50K']

le = LabelEncoder()

print('train data:')
print(le.fit_transform(ytoy_train))
print()
print('test data:')
print(le.transform(ytoy_test))

train data:
[1 1 1 0 0 1]

test data:
[0 1 1 0]


In [28]:
# How and when to do preprocessing
    # Apply transformer.fit ONLY on your TRAINING data.
    # **Fit on the training set, then transform on validation and test sets.**
    # Fit should not go on the entire dataset because it will impact how the training and validation sets are transformed
        # only fit on training set once
        # test set might include outliers, so if you fit this too it will influence it
        # test set always needs to be separate from the train and validation sets
        
    # do not apply fit_transform on all three, separately
        # The relative position of the points will change because the min max on test will be different from the min
        # max on the training set
    